In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv("employee_data.csv")

# ============================================================
# ✅ STEP 1: DATA CLEANING
# ============================================================

# -------------------------
# 1.1 HANDLE MISSING VALUES
# -------------------------
print("Missing values before cleaning:\n")
print(df.isnull().sum())

# Numeric columns imputation
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Experience_Years'] = df['Experience_Years'].fillna(df['Experience_Years'].median())
df['Salary_USD'] = df['Salary_USD'].fillna(df['Salary_USD'].mean())
df['Bonus_%'] = df['Bonus_%'].fillna(df['Bonus_%'].median())
df['Performance_Rating'] = df['Performance_Rating'].fillna(df['Performance_Rating'].mode()[0])
df['Overtime_Hours_Monthly'] = df['Overtime_Hours_Monthly'].fillna(df['Overtime_Hours_Monthly'].median())
df['Projects_Handled'] = df['Projects_Handled'].fillna(df['Projects_Handled'].mode()[0])

# Categorical columns imputation
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

print("\nMissing values after cleaning:\n")
print(df.isnull().sum())


# -------------------------
# 1.2 FIX NOISY / INVALID VALUES
# -------------------------

# Age outside human range
df.loc[(df['Age'] < 18) | (df['Age'] > 65), 'Age'] = np.random.randint(18, 66)

# Experience outside possible range
df.loc[(df['Experience_Years'] < 0) | (df['Experience_Years'] > 45), 'Experience_Years'] = \
    np.random.uniform(0, 45, size=df[(df['Experience_Years'] < 0) | (df['Experience_Years'] > 45)].shape[0])

# Bonus exceeding typical corporate % range
df.loc[df['Bonus_%'] > 50, 'Bonus_%'] = df['Bonus_%'].median()

# Unrealistic overtime values
df.loc[df['Overtime_Hours_Monthly'] > 100] = df['Overtime_Hours_Monthly'].median()

# Fix invalid dates
df['Joining_Date'] = pd.to_datetime(df['Joining_Date'], errors='coerce')
df['Joining_Date'] = df['Joining_Date'].fillna(df['Joining_Date'].mode()[0])

# Fix INR salaries accidentally mixed with USD
median_salary = df['Salary_USD'].median()
df.loc[df['Salary_USD'] > 20000, 'Salary_USD'] /= 80
df.loc[df['Salary_USD'] <= 0, 'Salary_USD'] = median_salary

print("\n✅ Noisy values cleaned.\n")
print(df.describe(include='all'))


# -------------------------
# 1.3 REMOVE DUPLICATES
# -------------------------
print("\nDuplicate rows before removal:", df.duplicated().sum())
df = df.drop_duplicates()
print("Duplicate rows after removal:", df.duplicated().sum())
print("Dataset shape:", df.shape)


# -------------------------
# 1.4 OUTLIER HANDLING (IQR)
# -------------------------

numeric_cols = df.select_dtypes(include=[np.number]).columns

def treat_outliers_iqr(column):
    Q1, Q3 = df[column].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR

    print(f"\n--- {column} ---")
    print("Lower Bound:", lower)
    print("Upper Bound:", upper)
    print("Outliers before:", ((df[column] < lower) | (df[column] > upper)).sum())

    df[column] = np.clip(df[column], lower, upper)

    print("Outliers after:", ((df[column] < lower) | (df[column] > upper)).sum())

for col in numeric_cols:
    treat_outliers_iqr(col)

print("\n✅ Outlier treatment completed.")


# ============================================================
# ✅ STEP 2: DATA INTEGRATION
# ============================================================

# 2.1 Standardize column names (lowercase for consistency)
df.columns = df.columns.str.lower()

# 2.2 Fix remaining INR salaries (redundant check for safety)
df.loc[df['salary_usd'] > 20000, 'salary_usd'] /= 80

print("\n✅ Data Integration complete.")


# ============================================================
# ✅ STEP 3: DATA TRANSFORMATION
# ============================================================

# -------------------------
# 3.1 ENCODING
# -------------------------

# Label Encoding for binary or ordinal columns
label_enc_cols = ['education_level', 'remote_work', 'attrition', 'gender']

for col in label_enc_cols:
    df[col] = df[col].astype(str)

le = LabelEncoder()
for col in label_enc_cols:
    df[col] = le.fit_transform(df[col])

print("✅ Label Encoding done.")

# One-Hot Encoding for nominal categories
one_hot_cols = ['department', 'job_role', 'city', 'state', 'country']
df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)

print("✅ One-Hot Encoding done.")


# -------------------------
# 3.2 SCALING (MinMax + Z-score)
# -------------------------

scaled_cols = ['age', 'experience_years', 'salary_usd', 'bonus_%',
               'overtime_hours_monthly', 'projects_handled']

# MinMax (0–1)
mm = MinMaxScaler()
df[scaled_cols] = mm.fit_transform(df[scaled_cols])

# Standardization
ss = StandardScaler()
df[scaled_cols] = ss.fit_transform(df[scaled_cols])

print("✅ Scaling completed.")


# -------------------------
# 3.3 LOG TRANSFORMATION on skewed features
# (After scaling, ensure values > -1)
# -------------------------

log_cols = ['salary_usd', 'bonus_%']

for col in log_cols:
    # Ensure valid input for log1p
    safe_values = np.where(df[col] > -1, df[col], -0.999999)
    df[f'log_{col}'] = np.log1p(safe_values)

print("✅ Log transformation done.")


# -------------------------
# 3.4 DISCRETIZATION (BINNING)
# -------------------------

df['salary_bin'] = pd.qcut(df['salary_usd'], q=3, labels=['Low', 'Medium', 'High'])

print("✅ Discretization done.")


# Remove columns not useful for modeling
columns_to_drop = ['name', 'joining_date']
df = df.drop(columns=[c for c in columns_to_drop if c in df.columns], errors='ignore')


# ============================================================
# ✅ STEP 4: DATA REDUCTION
# ============================================================

# -------------------------
# 4.1 PCA on selected continuous features
# -------------------------

pca_cols = ['age', 'experience_years', 'salary_usd', 'bonus_%',
            'overtime_hours_monthly', 'projects_handled']

pca_input = df[pca_cols].fillna(df[pca_cols].mean())

pca = PCA(n_components=0.95)
pca_features = pca.fit_transform(pca_input)

df_pca_final = pd.DataFrame(pca_features,
                            columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])

print("\n✅ PCA Completed Successfully!")
print(df_pca_final.head())

Missing values before cleaning:

Employee_ID                 0
Name                        0
Age                       286
Gender                     34
Department                  0
Experience_Years          273
Education_Level             0
Salary_USD                 58
Bonus_%                     0
Job_Role                    0
Joining_Date                0
City                        0
State                       0
Country                     0
Performance_Rating        207
Remote_Work               374
Overtime_Hours_Monthly    365
Projects_Handled            0
Attrition                   0
dtype: int64

Missing values after cleaning:

Employee_ID               0
Name                      0
Age                       0
Gender                    0
Department                0
Experience_Years          0
Education_Level           0
Salary_USD                0
Bonus_%                   0
Job_Role                  0
Joining_Date              0
City                      0
State          